### Google Drive requirements:
1. `CS-COCO.zip` dataset file
2. `adain` package
3. `content.jpg` and `style.jpg` images (must be the same size)
4. `vgg19-norm.pth` file with VGG19 weights
5. `checkpoints/checkpoint-<iter_num>.pth` (if you want to start training from checkpoint)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import sys

path2drive = '/content/drive/MyDrive'
zip_file = os.path.join(path2drive, 'CS-COCO.zip')
path2vgg = os.path.join(path2drive, 'vgg19-norm.pth')

sys.path.append(path2drive)

In [ ]:
from adain.train import *

import zipfile

import torch

In [ ]:
config = TrainingConfig(
    path2ds='/content/CS-COCO',
    checkpoints_dir=os.path.join(path2drive, 'checkpoints'),
    content_img=os.path.join(path2drive, 'content.jpg'),
    style_img=os.path.join(path2drive, 'style.jpg'),
    logs_dir=os.path.join(path2drive, 'logs'),
    batch_size=32,
    save_freq=125,
)

In [ ]:
archive = zipfile.ZipFile(zip_file, 'r')
archive.extractall(config.path2ds)

In [ ]:
if not os.path.exists(config.checkpoints_dir):
    os.makedirs(config.checkpoints_dir)

if not os.path.exists(config.logs_dir):
    os.makedirs(config.logs_dir)

In [ ]:
model = StyleTransferNetwork(path2vgg).to(DEVICE)

indices = [int(f[11:-4]) for f in os.listdir(config.checkpoints_dir)] + [-1, 0]
start = max(indices)

if start:
    path = os.path.join(config.checkpoints_dir, f'checkpoint-{start}.pth')
    state_dict = torch.load(path, map_location=DEVICE)
    model.load_state_dict(state_dict)

In [ ]:
for iter_num in train(model, config, start):
    save_model(model, config, iter_num)
    save_log(model, config, iter_num)